In [14]:
import pandas as pd 
import numpy as np

In [19]:
circuito = pd.read_csv('circuito_senoidal.csv', sep=',')

#transformando o ramo em índice
circuito.set_index('Ramo', drop=True, inplace=True)
circuito

,nó saída,nó chegada,R(Ohm),Xl(Ohm),Xc(Ohm),Vind(V),Fase_Vind,Iind(A),Fase_Iind
Ramo,,,,,,,,,
1,1,3,3,0,0,0,0,1,40
2,2,1,0,0,2,0,0,0,0
3,2,3,0,4,0,0,0,0,0
4,2,3,6,0,0,12,0,0,0


## Matriz Incidência (A)

Sabemos que a matriz tem dimensões nxb, aonde n representa a quantidade de nós e b a quantidade de ramos (branchs) 

In [24]:
# - Quantos nós e quantos ramos existem no circuito?

#criando um dataframe que possui somente os nós
nos = circuito[['nó saída', 'nó chegada']]

#sabemos que a quantidade de nós vai ser dada pelo máximo número encontrado na tabela "nos"
quantidade_de_nos = nos.values.max()

#a quantidade de ramos é dada pela quantidade de linhas que possuimos na tabela.
quantidade_de_ramos = circuito.shape[0] 

#a matriz de incidencia completa terá dimensões quantidade_de_nos x quantidade_de_ramos
incidencia_completa = np.zeros((quantidade_de_nos, quantidade_de_ramos)) #matriz de zeros


Ainda sabemos que como os nossos ramos são grafos orientados, a matriz de incidência terá em uma coluna somente 2 linhas com valores diferente de 0, uma com valor 1 (linha referente ao nó de saída) e outra com valor -1 (linha referente ao nó de chegada)

In [27]:
#Além disso terá 1 quando a corrente tiver saindo do nó e -1 quando estiver chegando
for ramo in range(quantidade_de_ramos): 
    #contando o nó a partir do zero
    no_de_saida = nos['nó saída'][ramo+1] - 1 # -1 dado que arrays indexa a partir do 0
    no_de_chegada =  nos['nó chegada'][ramo+1] - 1 

    #a matriz tem 1 na linha do nó de saida, coluna do ramo 
    incidencia_completa[no_de_saida,ramo] = 1
#a matriz tem -1 na linha do nó de chegada, coluna do ramo
    incidencia_completa[no_de_chegada, ramo] = -1

incidencia_completa

array([[ 1., -1.,  0.,  0.],
       [ 0.,  1.,  1.,  1.],
       [-1.,  0., -1., -1.]])

Porém, sabemos que a matriz de incidência é linearmente dependente, então não precisamos usar todas as linhas da matriz, podemos alcançar todos os resultados desejados com a matriz de incidência reduzida (retirando a última linha)

In [29]:
# "A" será a matriz de incidência reduzida. Iremos retirar a última linha da incidencia_completa
A = incidencia_completa[:-1,:]
A

array([[ 1., -1.,  0.,  0.],
       [ 0.,  1.,  1.,  1.]])

In [30]:
#a transpota da matriz de incidência reduzida:
A.T

array([[ 1.,  0.],
       [-1.,  1.],
       [ 0.,  1.],
       [ 0.,  1.]])

## Matriz admitância (Yb)

#### Para calcular a matriz de admitância vamos realizar os seguintes passos:
1. Montar uma tabela com as reatâncias resultantes de cada ramo 
2. Aplicar a função inversa e obter as admitâncias resultantes de cada ramo
3. Montar a matriz diagonal "Yb" a partir da tabela construída

In [36]:
#Temos que a reatância de cada ramo pode ser calculada da seguinte forma:
R = circuito['R(Ohm)']
Xc = circuito['Xc(Ohm)']
Xl = circuito['Xl(Ohm)']
Zeq = R + 1j*(Xc - Xl)

Zeq

Ramo
1    3.000000+0.000000j
2    0.000000+2.000000j
3    0.000000-4.000000j
4    6.000000+0.000000j
dtype: complex128

In [41]:
#Então a admitância:
Yeq = 1/Zeq
Yeq

Ramo
1    0.333333+0.000000j
2    0.000000-0.500000j
3   -0.000000+0.250000j
4    0.166667+0.000000j
dtype: complex128

In [45]:
Yb = np.diag(Yeq)
#apresentando de forma mais legível
print(Yb, sep='\n')

[[ 0.33333333+0.j    0.        +0.j    0.        +0.j    0.        +0.j  ]
 [ 0.        +0.j    0.        -0.5j   0.        +0.j    0.        +0.j  ]
 [ 0.        +0.j    0.        +0.j   -0.        +0.25j  0.        +0.j  ]
 [ 0.        +0.j    0.        +0.j    0.        +0.j    0.16666667+0.j  ]]
